# Import Libraries

In [29]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import string as st
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import torch
import fasttext
from sklearn.decomposition import PCA
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

# Linguistic Modules

In [11]:
def preprocess_text(text):
    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
    # Remove stopwords, punctuation, and lemmatize
    tokens = [token.lemma_ for token in doc 
             if not token.is_stop and not token.is_punct and token.is_alpha]
    
    return ' '.join(tokens)

In [12]:
def extract_linguistic_features(text):
    doc = nlp(text)
    features = {
        'num_verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'num_nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'sentence_length': len(doc),
        'blooms_verb_present': any(token.text in {'analyze', 'evaluate', 'create'} for token in doc)
    }
    return features

In [13]:
def get_sentence_vector(text):
    words = text.strip().split()
    vectors = [model.get_word_vector(w) for w in words]
    if not vectors:
        return np.zeros(300)
    return np.mean(vectors, axis=0)

# Pre-Processing

### Import Data

In [34]:
label_mapper = {
    'BT1' : 'knowledge',
    'BT2' : 'comprehension',
    'BT3' : 'application',
    'BT4' : 'analysis',
    'BT5' : 'synthesis',
    'BT6' : 'evaluation'
}

# Load dataset
df = pd.DataFrame()
for i in range(1,5):
    q_df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/dataset' + str(i) + '.csv')
    df = pd.concat([df , q_df])

# Apply preprocessing
mask = df['label'].isin(label_mapper.keys())
df['label'] = df['label'].mask(mask, df['label'].map(label_mapper))
df['label'] = df['label'].str.lower()

df['processed_question'] = df['question'].apply(preprocess_text)

## Tokenize

BERT model is used to retain context

In [ ]:
# Load pre-trained BERT (TensorFlow version)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode
inputs = tokenizer(df['question'].tolist(), return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
print("BERT Sentence Embedding Shape:", sentence_embedding.shape)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
pca = PCA(n_components=100)
reduced_embeddings = pca.fit_transform(sentence_embedding.numpy())


### TF-IDF 
Execute either BERT or IF-IDF

In [84]:
vectorizer = TfidfVectorizer(max_features= 1000)
tfidf = vectorizer.fit_transform(df['processed_question'])

# Convert TF-IDF sparse matrix to DataFrame with appropriate column names
tfidf_df = pd.DataFrame(tfidf.toarray(), columns= vectorizer.get_feature_names_out(), index=df.index)

Execute if linguistic features like count for verb,noun,etc needed

In [80]:
# Adding linguistic features
ling_features = df['processed_question'].apply(extract_linguistic_features).apply(pd.Series)

# Concatenate DataFrames
token_df = pd.concat([ling_features, tfidf_df], axis=1)

# Drop unnecessary columns (if needed)
token_df = token_df.drop(columns=['blooms_verb_present'], axis=1)
token_df.columns = token_df.columns.map(str)

# token_df = pca.fit_transform(token_df)

# Feature Extraction

### Label Balancing

In [85]:
x , y = tfidf_df , df['label']
x_resampled , y_resampled = SMOTETomek().fit_resample(x , y)

### Dimensionality Reduction

# Modelling

## Basic Machine Leanring

In [86]:
x_train , x_test , y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=34 , stratify= y_resampled)

### Random Forest

In [87]:
params = {
    'n_estimators' : [2 , 5 , 10 , 20 , 50 , 100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 20 , 50 , 100 , 200 , 350 , 500]
}
param_search = GridSearchCV(estimator= RandomForestClassifier(), param_grid= params, cv=5)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
107,0.793406,"{'criterion': 'entropy', 'max_depth': 500, 'n_estimators': 100}"
59,0.792196,"{'criterion': 'entropy', 'max_depth': None, 'n_estimators': 100}"
47,0.791954,"{'criterion': 'gini', 'max_depth': 350, 'n_estimators': 100}"
101,0.791712,"{'criterion': 'entropy', 'max_depth': 350, 'n_estimators': 100}"
5,0.790502,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 100}"
95,0.790442,"{'criterion': 'entropy', 'max_depth': 200, 'n_estimators': 100}"
58,0.790260,"{'criterion': 'entropy', 'max_depth': None, 'n_estimators': 50}"
100,0.790200,"{'criterion': 'entropy', 'max_depth': 350, 'n_estimators': 50}"
106,0.789655,"{'criterion': 'entropy', 'max_depth': 500, 'n_estimators': 50}"
46,0.789232,"{'criterion': 'gini', 'max_depth': 350, 'n_estimators': 50}"


In [88]:
classifier = RandomForestClassifier(n_estimators= 100 , max_depth= 500 , criterion= 'gini')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.87      0.84      0.86       562
  application       0.90      0.88      0.89       564
comprehension       0.82      0.86      0.84       551
   evaluation       0.78      0.76      0.77       541
    knowledge       0.76      0.78      0.77       547
    synthesis       0.75      0.75      0.75       541

     accuracy                           0.81      3306
    macro avg       0.81      0.81      0.81      3306
 weighted avg       0.81      0.81      0.81      3306



In [78]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
tfidf_q = pd.DataFrame(tfidf_q.toarray(), columns=vectorizer.get_feature_names_out())



# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, tfidf_q], axis=1)
token_q = token_q.drop(columns= ['blooms_verb_present'])

print(classifier.predict(token_q))

['application']


In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
inputs =  tokenizer([question], return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]